<a href="https://colab.research.google.com/github/esul-gif/tom-jerry/blob/main/Tom_and_Jerry_ML_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Install required packages
!pip install scikit-image
!pip install numpy
!pip install pandas

# Now import them
from skimage import io, color
from skimage.feature import hog
from skimage.transform import resize
import numpy as np
import pandas as pd


In [19]:
gt = pd.read_csv('ground_truth.csv')
dataset_path = '/content/dataset/Euc'

In [20]:
#Function for processing image into HOG

def process_image(image_path, output_shape=(128,128)):
    image = io.imread(image_path)
    if len(image.shape) == 3:
        image = color.rgb2gray(image) #greyscale so it works better with HOG
    image = resize(image,output_shape, anti_aliasing=True)
    #extract hog feature
    feature_desc = hog(image, orientations= 8, pixels_per_cell=(16,16), cells_per_block=(1,1), visualize = False, feature_vector=True)

    return feature_desc




In [21]:
features = []
labels = []

#process each image

for index, row in gt.iterrows():
    image_path = 'tom_and_jerry/' + row['filename']
    img_features = process_image(image_path)
    features.append(img_features)

    labels.append([row['tom'],row['jerry']])

In [22]:
from sklearn.model_selection import train_test_split

# Convert features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)  # Multi-label scenario

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [23]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

# Train a classifier
model = OneVsRestClassifier(SVC(kernel='linear'))
model.fit(X_train, y_train)


OneVsRestClassifier(estimator=SVC(kernel='linear'))

In [25]:
from sklearn.metrics import accuracy_score

predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 58.85%
